In [4]:
from IPython.display import display, HTML , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
from io import BytesIO
import matplotlib.pyplot as plt
from numpy import random
import pandas as pd
import numpy as np
import logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../src/python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../'
config = L.get_config(PROJECT_HOME_DIR)
session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(session == None):
   raise Exception(f'Unable to connect to snowflake. Validate connection information ')

session.use_role(f'''{config['APP_DB']['role']}''')
session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df1 = session.sql('select current_account(), current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df1)

### Initialization

,CURRENT_ACCOUNT(),CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,CIB92733,JPRUSA,SERVICESNOW_USER_ROLE,FASHION,PUBLIC


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

In [6]:
X, y = load_iris(return_X_y=True)

In [7]:
clf = RandomForestClassifier()
clf.fit(X, y)

RandomForestClassifier()

In [8]:
import importlib
from snowflake.ml.registry import model_registry

In [10]:
importlib.reload(model_registry)
registry_name = "model_registry_jprusa"

In [14]:
# Create a new model registry. This will be a no-op if the registry already exists.
create_result = model_registry.create_model_registry(session=session)

In [18]:
registry = model_registry.ModelRegistry(session=session)

In [19]:
registry

In [20]:
help(registry.log_model)

Help on method log_model in module snowflake.ml.registry.model_registry:

log_model(model_name: str, model_version: str, *, model: Any, description: Optional[str] = None, tags: Optional[Dict[str, str]] = None, conda_dependencies: Optional[List[str]] = None, pip_requirements: Optional[List[str]] = None, signatures: Optional[Dict[str, snowflake.ml.model._signatures.core.ModelSignature]] = None, sample_input_data: Optional[Any] = None, code_paths: Optional[List[str]] = None, options: Optional[snowflake.ml.model.type_hints.BaseModelSaveOption] = None) -> str method of snowflake.ml.registry.model_registry.ModelRegistry instance
    Uploads and register a model to the Model Registry.
    
    Args:
        model_name: The given name for the model. The combination (name + version) must be unique for each model.
        model_version: Version string to be set for the model. The combination (name + version) must be unique for
            each model.
        model: Local model object in a suppor

In [52]:
get_current_date()

'12:28:58__8_22_2023'

In [53]:
registry = model_registry.ModelRegistry(session=session)
model_id = registry.log_model(model=clf, model_name="test_model", model_version="12:28:58__8_22_2023", tags={
"stage": "testing", "classifier_type": "RandomForestClassifer"}, sample_input_data=X)

/Users/jprusa/miniconda3/snowpark-ml/lib/python3.10/site-packages/snowflake/ml/model/model_signature.py:48: UserWarning: The sample input has 150 rows, thus a truncation happened before inferring signature. This might cause inaccurate signature inference. If that happens, consider specifying signature manually.
  warnings.warn(
/Users/jprusa/miniconda3/snowpark-ml/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:910: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [54]:
# The object API can be used to reference a model after creation.
model = model_registry.ModelReference(
    registry=registry, 
    model_name="test_model", 
    model_version="12:28:58__8_22_2023"
)
print("Registered new model:", model_id)

Registered new model: c8be0ccc411911ee913646c0d41e10b8


In [57]:
model_list.select("NAME","VERSION","TAGS","METRICS").show()

--------------------------------------------------------------------------------------------------------------------
|"NAME"                          |"VERSION"            |"TAGS"                                         |"METRICS"  |
--------------------------------------------------------------------------------------------------------------------
|test_model                      |101                  |{                                              |NULL       |
|                                |                     |  "classifier_type": "RandomForestClassifer",  |           |
|                                |                     |  "stage": "testing"                           |           |
|                                |                     |}                                              |           |
|test_model                      |12:28:58__8_22_2023  |{                                              |NULL       |
|                                |                     |  "class

In [31]:
import datetime

In [50]:
def get_current_date():
    current_date=datetime.datetime.now()
    year = current_date.year
    month = current_date.month
    day = current_date.day
    hour = current_date.hour
    minute = current_date.minute
    second = current_date.second

    return f"{hour}:{minute}:{second}__{month}_{day}_{year}"
    
    

'12:27:35__8_22_2023'